# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
# Read in clean_weather_data.csv
weather_data_filepath = "resource/clean_weather_data.csv"
weather_data = pd.read_csv(weather_data_filepath, index_col = 0)
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,atuona,-9.8000,-139.0333,298.90,75,0,7.32,PF,1644522253
1,puerto ayora,-0.7393,-90.3518,302.13,85,96,3.13,EC,1644522254
2,punta arenas,-53.1500,-70.9167,285.16,57,40,12.86,CL,1644522254
3,birigui,-21.2886,-50.3400,302.20,54,72,3.13,BR,1644522121
4,rikitea,-23.1203,-134.9692,299.38,72,89,8.41,PF,1644522255
...,...,...,...,...,...,...,...,...,...
551,paamiut,61.9940,-49.6678,262.71,73,100,3.97,GL,1644523232
552,lagoa,39.0500,-27.9833,287.91,73,100,2.46,PT,1644523233
553,isangel,-19.5500,169.2667,299.76,90,100,10.25,VU,1644523233
554,dzaoudzi,-12.7887,45.2699,298.09,94,100,1.54,YT,1644523234


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key = g_key)

locations = weather_data[["Lat", "Lng"]]
weight = weather_data["Humidity"].astype(float)

In [7]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = weight, 
                                 dissipating = False, max_intensity = 10,
                                 point_radius = 1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Condition
# - A max temperature lower than 80 degrees (294.2611 Kelvin) but higher than 70 (299.8167 Kelvin)
# - Wind speed less than 10 mph (4.4705 m/s)
# - Zero cloudiness
# - Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal
# Adding a condition
# - Needs to be in US

good_weather_city = weather_data.loc[(((weather_data["Max Temp"] > 294.2611) & (weather_data["Max Temp"] < 299.8167)) & (weather_data["Wind Speed"] < 4.4705) & (weather_data["Cloudiness"] == 0) & (weather_data["Country"] == "US")), :]
good_weather_city


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
108,fortuna,40.5982,-124.1573,298.07,52,0,0.00,US,1644522435
128,pacific grove,36.6177,-121.9166,298.60,41,0,1.79,US,1644522447
140,kahului,20.8947,-156.4700,298.62,64,0,0.00,US,1644522454
142,kapaa,22.0752,-159.3190,297.86,74,0,2.57,US,1644522455
199,eureka,40.8021,-124.1637,298.22,54,0,1.19,US,1644522612
265,san joaquin,37.9333,-121.3011,296.75,48,0,2.06,US,1644522710
276,laguna,38.4210,-121.4238,296.60,58,0,0.89,US,1644522714
350,makakilo city,21.3469,-158.0858,297.75,67,0,0.00,US,1644522877
454,stephenville,32.2207,-98.2023,295.34,21,0,1.34,US,1644523055
516,ukiah,39.1502,-123.2078,299.62,34,0,0.40,US,1644523150


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = good_weather_city.copy()
hotel_df["Hotel Name"] = ""
hotel_df

params = {"radius": 50000, 
          "types": "hotel", 
          "keyword": "hotel",
          "key": g_key}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # print the name_address url, avoid doing for public github repos in order to avoid exposing key
    # print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


In [16]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address,Hotel Rating
108,fortuna,40.5982,-124.1573,298.07,52,0,0.00,US,1644522435,Holiday Inn Express & Suites Arcata/Eureka-Air...,"3107 Concorde Dr, McKinleyville",4.4
128,pacific grove,36.6177,-121.9166,298.60,41,0,1.79,US,1644522447,Hilton Garden Inn Monterey,"1000 Aguajito Rd, Monterey",4.1
140,kahului,20.8947,-156.4700,298.62,64,0,0.00,US,1644522454,Hotel Lanai,"828 Lanai Ave, Lanai City",4.6
142,kapaa,22.0752,-159.3190,297.86,74,0,2.57,US,1644522455,Kauai Beach Resort & Spa,"4331 Kauai Beach Dr, Lihue",4.3
199,eureka,40.8021,-124.1637,298.22,54,0,1.19,US,1644522612,Holiday Inn Express & Suites Arcata/Eureka-Air...,"3107 Concorde Dr, McKinleyville",4.4
265,san joaquin,37.9333,-121.3011,296.75,48,0,2.06,US,1644522710,Aloft Dublin-Pleasanton,"4075 Grafton St, Dublin",4.3
276,laguna,38.4210,-121.4238,296.60,58,0,0.89,US,1644522714,"Country Inn & Suites by Radisson, Dixon, CA - ...","155 Dorset Dr, Dixon",4.2
350,makakilo city,21.3469,-158.0858,297.75,67,0,0.00,US,1644522877,Waikiki Resort Hotel,"2460 Koa Ave, Honolulu",4.0
454,stephenville,32.2207,-98.2023,295.34,21,0,1.34,US,1644523055,"Holiday Inn Express & Suites Granbury, an IHG ...","1515 N Plaza Dr, Granbury",4.3
516,ukiah,39.1502,-123.2078,299.62,34,0,0.40,US,1644523150,Hotel Breakers,"39300 CA-1, Gualala",4.3


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [52]:
# Add marker layer ontop of heat map
hotel_list = hotel_df[["City", "Country", "Hotel Name"]]
marker_locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(marker_locations, info_box_content=[f"Hotel Info: City = {hotel_list.iloc[hotel, 0]}, Country = {hotel_list.iloc[hotel, 1]}, Hotel Name = {hotel_list.iloc[hotel, 2]}" for hotel in range(len(hotel_list))])
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))